In [1]:
username = "Fro116"
source = "MAL";

In [2]:
import NBInclude: @nbinclude
@nbinclude("../../InferenceAlphas/Alpha.ipynb");

┌ Warning: source_name not defined: using all/Inference
└ @ Main ~/Desktop/RecommenderSystem/notebooks/InferenceAlphas/Alpha.ipynb:In[+2]:3


In [3]:
coefs = [-5.481506, 1.2974571, -1.084365, 0.24383727]
coefs = exp.(coefs);

In [4]:
# function vec(x::DropItems)
#     m = zeros(num_items("manga") + num_items("anime") + 1)
#     for y in x.manga_items
#         m[y] = 1
#     end
#     for y in x.anime_items
#         m[y+num_items("manga")] = 1
#     end
#     m[end] = 1 # the intercept term
#     m
# end;

In [5]:
# @memoize function get_values(medium, task, coefs)
#     relu(x) = max(x, zero(eltype(x)))
#     imp_alpha = read_recommendee_params("$medium/$task/Transformer/v1/implicit")["alpha"]
#     exp_alpha = read_recommendee_params("$medium/$task/Transformer/v1/explicit")["alpha"]
#     exp_alpha = relu.((exp_alpha .+ 10) ./ 10)
#     utility = sum(coefs .* [imp_alpha, exp_alpha, log.(imp_alpha), log.(exp_alpha)])

#     noise =
#         read_recommendee_params("anime/temporal_causal/Transformer/v1/implicit")["noise"]
#     idxs = [x for x in 1:length(noise) if typeof(noise[x]) == DropItems]
#     noise = noise[idxs]
#     utility = utility[:, idxs]

#     utility, noise
# end

In [6]:
function encode(x::DropItems)
    m = zeros(num_items("manga") + num_items("anime") + 1)
    for y in x.manga_items
        m[y] = -1
    end
    for y in x.anime_items
        m[y+num_items("manga")] = -1
    end
    m[end] = 1 # the intercept term
    m
end

@memoize function get_attribution_data(medium, task, coefs)
    relu(x) = max(x, zero(eltype(x)))
    imp_alpha = read_recommendee_params("$medium/$task/Transformer/v1/implicit")["alpha"]
    exp_alpha = read_recommendee_params("$medium/$task/Transformer/v1/explicit")["alpha"]
    exp_alpha = relu.((exp_alpha .+ 10) ./ 10)
    utility = sum(coefs .* [imp_alpha, exp_alpha, log.(imp_alpha), log.(exp_alpha)])

    noise =
        read_recommendee_params("anime/temporal_causal/Transformer/v1/implicit")["noise"]
    idxs = [x for x in 1:length(noise) if typeof(noise[x]) == DropItems]
    noise = noise[idxs]
    utility = utility[:, idxs]

    M = reduce(hcat, encode.(noise))
    utility, M
end

get_attribution_data (generic function with 1 method)

In [13]:
utility, M = get_attribution_data("anime", "temporal_causal", coefs)

([1.6782465136167606 1.1710987052889827 … 1.1594172754168972 0.41184471372214115; 2.249898711298231 1.9227066145592775 … 2.096234981367689 1.9163803710718277; … ; 2.3933555562967843 2.0348550368410763 … 1.6707101750712914 1.6797779066593121; 0.5470715110167559 1.1444750355613134 … 0.7504151946766187 0.019771798629687698], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 1.0 1.0 … 1.0 1.0])

In [14]:
kanojo = 22311 # kanojo
bunny = 19765 # bunny
bocchi = 6437 # spy x fam
ojamajo = 5802 # clear card
touch = 23762 # kodocha
fullmoon = 8706 # kodocha
tengoku = 5667
item = bocchi + 1;

In [22]:
attrs = M' \ utility[item, :];

In [35]:
M' \ utility[1:2, :]'

91822×2 Matrix{Float64}:
 0.0          0.0
 0.0          0.0
 0.0          0.0
 0.0          0.0
 0.0          0.0
 0.0          0.0
 0.0          0.0
 0.0          0.0
 0.0          0.0
 0.0          0.0
 0.0          0.0
 0.0          0.0
 0.0          0.0
 ⋮           
 0.0          0.0
 0.0          0.0
 0.0184192   -0.0174045
 0.0          0.0
 0.0          0.0
 0.0          0.0
 0.0          0.0
 0.0          0.0
 0.0          0.0
 0.0          0.0
 0.0          0.0
 0.00672504   0.0104289

In [33]:
utility[1:2, :]'

256×2 adjoint(::Matrix{Float64}) with eltype Float64:
 1.67825   2.2499
 1.1711    1.92271
 0.703921  2.06481
 1.49094   2.45551
 0.721562  2.14207
 0.655485  2.17063
 1.85532   2.56885
 0.35367   1.42878
 1.13662   2.33534
 0.946271  2.15146
 1.91048   2.5347
 1.21162   1.68324
 0.910594  2.01299
 ⋮         
 1.50398   2.42999
 1.25361   2.52799
 0.581859  1.80086
 1.05393   1.95093
 1.74468   2.33743
 1.45902   2.0407
 1.25016   2.57782
 1.11329   2.00038
 1.0881    2.27063
 0.844142  2.04134
 1.15942   2.09623
 0.411845  1.91638

In [28]:
sortperm(attrs2) .- num_items("manga") .- 1

91822-element Vector{Int64}:
 -64755
   2096
 -21772
   1844
   1740
 -45896
 -44237
   2897
   3618
 -42394
    548
  -8234
 -64122
      ⋮
   3902
 -16983
 -43691
   1951
 -58287
 -58825
 -31511
  -3397
  -7509
   1735
 -44160
 -16414

In [27]:
sortperm(attrs) .- num_items("manga") .- 1

91822-element Vector{Int64}:
   2486
  11782
  16537
   5876
 -22957
   8645
   1846
   9233
  -4804
  18065
   6309
 -64122
  13899
      ⋮
  15105
     44
  24737
  17460
  11586
 -55789
  11210
 -27781
  19010
    689
   2897
 -13875

In [12]:
sortperm(attrs) .- 1

LoadError: UndefVarError: `attrs` not defined